In [5]:
import pandas as pd
import json

In [227]:
df = pd.read_csv("scraped_contributor_information_for_repos.csv")

In [228]:
df.head()

,Unnamed: 0,repo,repo_url,avg_num_contributors,start_date,contrib_info,date_urls
0,0,H2O,https://github.com/h2oai/h2o-3/graphs/contribu...,9.6,"Mar 2, 2014",{'https://github.com/h2oai/h2o-3/graphs/contri...,['https://github.com/h2oai/h2o-3/graphs/contri...
1,1,Snorkel AI,https://github.com/snorkel-team/snorkel/graphs...,2.4,"Feb 21, 2016 - Feb 12, 2022",{'https://github.com/snorkel-team/snorkel/grap...,['https://github.com/snorkel-team/snorkel/grap...
2,2,kedro,https://github.com/kedro-org/kedro/graphs/cont...,9.2,"May 12, 2019 - Feb 12, 2022",{'https://github.com/kedro-org/kedro/graphs/co...,['https://github.com/kedro-org/kedro/graphs/co...
3,3,MLFlow,https://github.com/mlflow/mlflow/graphs/contri...,7.4,"Jun 03, 2018 - Feb 12, 2022",{'https://github.com/mlflow/mlflow/graphs/cont...,['https://github.com/mlflow/mlflow/graphs/cont...
4,4,Polyaxon,https://github.com/polyaxon/polyaxon/graphs/co...,2.4,"Mar 20, 2016 - Feb 12, 2022",{'https://github.com/polyaxon/polyaxon/graphs/...,['https://github.com/polyaxon/polyaxon/graphs/...


In [246]:
def extract_commit_info(row):
    """Takes a single row and returns a dataframe of contributions"""
    # read the contrib_info into a python dictionary
    repo = list(row["repo"])[0]
    contrib_info = ast.literal_eval(list(row["contrib_info"])[0])
    
    # extract the information about the scrape
    re_extract_data = r"(?<=github.com/)([^/]*)(/)([^/]*)(/graphs/contributors\?from=)([^\&]*)(\&to=)([^\&]*)"
    series_urls = pd.Series(contrib_info.keys())
    df_scrapeinfo = series_urls.str.extract(re_extract_data)
    df_scrapeinfo["repo"] = repo
    df_scrapeinfo.columns = df_scrapeinfo.columns.map(str)
    df_scrapeinfo.rename(columns={"0": "github_org", 
                                  "2": "github_proj",
                                  "4":"date_start", 
                                  "6":"date_end"}, 
                         inplace=True)
    df_scrapeinfo["scrape_url"] = series_urls
    df_scrapeinfo = \
        df_scrapeinfo[["repo","scrape_url","github_org","github_proj","date_start","date_end"]]\
        .reset_index(drop=True)
    
    # extract the contribution information
    data = []
    for repo_url, el in contrib_info.items():
        for user, contribution in el.items():
            data.append([repo_url,
                         user,
                         re.findall(r"(\d*)",contribution[0])[0], #commits
                         re.findall(r"(\d*)",contribution[1])[0], #lines_add
                         re.findall(r"(\d*)",contribution[2])[0]  #lines_del
                        ])
    df_contributions = \
        pd.DataFrame(data, columns=["scrape_url","user","commits","lines_add","lines_del"])\
        .reset_index(drop=True)
    
    # join together the information
    return(df_scrapeinfo.merge(df_contributions, on="scrape_url")[[
        "repo",
        "github_org",
        "github_proj",
        "date_start",
        "date_end",
        "user",
        "commits",
        "lines_add",
        "lines_del"
    ]])

In [254]:
#https://stackoverflow.com/questions/13050003/apply-function-to-pandas-dataframe-that-can-return-multiple-rows
df_contributions = \
    df\
    .groupby("repo", group_keys=False)\
    .apply(lambda df: extract_commit_info(df.iloc[[0]]))\
    .sort_values(by=["github_org","date_start","user"])\
    .reset_index(drop=True)

df_contributions.to_csv("2022-02-18 - contributions.csv", index=False)